# 0. 数据下载

In [1]:
!mkdir datasets
!curl -o /home/secretnote/workspace/datasets/bank.csv https://secretflow-data.oss-accelerate.aliyuncs.com/datasets/bank_marketing/bank.csv

mkdir: cannot create directory ‘datasets’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  450k  100  450k    0     0  1592k      0 --:--:-- --:--:-- --:--:-- 1592k


In [2]:
import warnings
import pandas as pd
warnings.filterwarnings("ignore",category=DeprecationWarning)
pd.options.mode.chained_assignment = None # default='warn'
df = pd.read_csv("~/workspace/datasets/bank.csv", sep=';')
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,services,married,secondary,no,-333,yes,no,cellular,30,jul,329,5,-1,0,unknown,no
4517,57,self-employed,married,tertiary,yes,-3313,yes,yes,unknown,9,may,153,1,-1,0,unknown,no
4518,57,technician,married,secondary,no,295,no,no,cellular,19,aug,151,11,-1,0,unknown,no
4519,28,blue-collar,married,secondary,no,1137,no,no,cellular,6,feb,129,4,211,3,other,no


In [3]:
alice_data = df[["age","job","marital","education","y"]]
alice_data

,age,job,marital,education,y
0,30,unemployed,married,primary,no
1,33,services,married,secondary,no
2,35,management,single,tertiary,no
3,30,management,married,tertiary,no
4,59,blue-collar,married,secondary,no
...,...,...,...,...,...
4516,33,services,married,secondary,no
4517,57,self-employed,married,tertiary,no
4518,57,technician,married,secondary,no
4519,28,blue-collar,married,secondary,no


In [4]:
bob_data = df[
    [
        "default",
        "balance",
        "housing",
        "loan",
        "contact",
        "day",
        "month",
        "duration",
        "campaign",
        "pdays" ,
        "previous",
        'poutcome'
    ]
]

bob_data

,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown
1,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure
2,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure
3,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown
4,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...
4516,no,-333,yes,no,cellular,30,jul,329,5,-1,0,unknown
4517,yes,-3313,yes,yes,unknown,9,may,153,1,-1,0,unknown
4518,no,295,no,no,cellular,19,aug,151,11,-1,0,unknown
4519,no,1137,no,no,cellular,6,feb,129,4,211,3,other


# 1. 数据处理

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
alice_data.loc[:,'job']= encoder.fit_transform(alice_data['job'])
alice_data.loc[:,'marital']= encoder.fit_transform(alice_data['marital'])
alice_data.loc[:,'education']=encoder.fit_transform(alice_data['education'])
bob_data.loc[:,'default']=encoder.fit_transform(bob_data['default'])
bob_data.loc[:,'housing']= encoder.fit_transform(bob_data['housing'])
bob_data.loc[:,'loan']=encoder.fit_transform(bob_data['loan'])
bob_data.loc[:,'contact']=encoder.fit_transform(bob_data['contact'])
bob_data.loc[:,'poutcome']= encoder.fit_transform(bob_data['poutcome'])
bob_data.loc[:,'month']= encoder.fit_transform(bob_data['month'])
train_label=encoder.fit_transform(alice_data['y'])
alice_data.drop(columns=["y"],inplace=True)
scaler = MinMaxScaler()
alice_data=scaler.fit_transform(alice_data)
bob_data=scaler.fit_transform(bob_data)

In [6]:
alice_data

array([[0.16176471, 0.90909091, 0.5       , 0.        ],
       [0.20588235, 0.63636364, 0.5       , 0.33333333],
       [0.23529412, 0.36363636, 1.        , 0.66666667],
       ...,
       [0.55882353, 0.81818182, 0.5       , 0.33333333],
       [0.13235294, 0.09090909, 0.5       , 0.33333333],
       [0.36764706, 0.18181818, 1.        , 0.66666667]])

# 2. 本地模拟

## 2.1 明文模型搭建

In [7]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from sklearn.model_selection import train_test_split

class SplitMLP(tf.keras.Model):
    def __init__ (self):
        super(SplitMLP,self).__init__()
        self.alice_model= keras.Sequential(
            [
                layers.Dense(100,activation="relu"),
                layers.Dense(32,activation="relu")
            ]
        )
        self.bob_model=keras.Sequential(
            [
                layers.Dense(100, activation="relu"),
                layers.Dense(32,activation="relu")
            ]
        )
        self.fuse_layer=layers.Dense(64,activation='relu')
        self.output_layer= layers.Dense(1,activation='sigmoid')
    def __call__(self,inputs):
        alice_feature = inputs['alice_feature']
        bob_feature = inputs['bob_feature']
        # hiddens
        h_alice=self.alice_model(alice_feature)
        h_bob= self.bob_model(bob_feature)
        merged_layer=layers.concatenate([h_alice,h_bob])
        fuse_layer_out=self.fuse_layer(merged_layer)
        output=self.output_layer(fuse_layer_out)
        return output
# Assuming you have imported alice_data, bob_data, and train_label properly
# Convert Pandas DataFrame to NumPy arrays
# Define the input layers
alice_input= tf.keras.Input(shape=(4,),name='alice_feature')
bob_input =tf.keras.Input(shape=(12,),name='bob_feature')
split_mlp=SplitMLP()
logits= split_mlp({'alice_feature': alice_input, 'bob_feature': bob_input})
model= tf.keras.Model(
    inputs=[alice_input, bob_input],
    outputs=logits
)
model.compile(
    loss="binary_crossentropy",
    optimizer='adam',
    metrics=[tf.keras.metrics.AUC()]
)
train_dataset=tf.data.Dataset.from_tensor_slices(
    (
        {
            "alice_feature": alice_data,
            "bob_feature": bob_data
        },
        train_label,
    )
).batch(64)
model.fit(train_dataset,epochs=10)

2024-08-06 09:04:56.287240: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-06 09:04:56.288714: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-06 09:04:56.314608: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-06 09:04:56.315587: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-06 09:04:57.065160: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

Epoch 1/10


2024-08-06 09:04:58.318199: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype int64 and shape [4521]
	 [[{{node Placeholder/_2}}]]


71/71 [==============================] - 1s 2ms/step - loss: 0.3936 - auc: 0.5684
Epoch 2/10
71/71 [==============================] - 0s 1ms/step - loss: 0.3222 - auc: 0.7421
Epoch 3/10
71/71 [==============================] - 0s 1ms/step - loss: 0.2894 - auc: 0.8191
Epoch 4/10
71/71 [==============================] - 0s 1ms/step - loss: 0.2639 - auc: 0.8555
Epoch 5/10
71/71 [==============================] - 0s 1ms/step - loss: 0.2557 - auc: 0.8659
Epoch 6/10
71/71 [==============================] - 0s 1ms/step - loss: 0.2518 - auc: 0.8708
Epoch 7/10
71/71 [==============================] - 0s 1ms/step - loss: 0.2488 - auc: 0.8750
Epoch 8/10
71/71 [==============================] - 0s 2ms/step - loss: 0.2462 - auc: 0.8782
Epoch 9/10
71/71 [==============================] - 0s 1ms/step - loss: 0.2441 - auc: 0.8810
Epoch 10/10
71/71 [==============================] - 0s 1ms/step - loss: 0.2421 - auc: 0.8831


In [8]:
import secretflow as sf
sf.shutdown( )
sf.init(['alice','bob'],address='local')
alice,bob=sf.PYU('alice'),sf.PYU('bob')

/usr/local/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-08-06 09:05:01,858	WARNING services.py:1996 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=4.23gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2024-08-06 09:05:01,992	INFO worker.py:1724 -- Started a local Ray instance.


In [9]:
from secretflow.data.split import train_test_split
from secretflow.ml.nn import SLModel

## 2.2 创建联邦表

In [10]:
spu=sf.SPU(sf.utils.testing.cluster_def(['alice','bob']))
from secretflow.utils.simulation.datasets import load_bank_marketing
# Alice has the first four features,
# while bob has the left features
# data是联邦表
data =load_bank_marketing(parts={alice:(0,4),bob:(4,16)},axis=1)
# Alice holds the label.
label = load_bank_marketing(parts={alice:(16,17)},axis=1)

INFO:root:Create proxy actor <class 'secretflow.data.core.agent.PartitionAgent'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.data.core.agent.PartitionAgent'> with party bob.
INFO:root:Create proxy actor <class 'secretflow.data.core.agent.PartitionAgent'> with party alice.


## 2.3 联邦表数据处理

In [11]:
from secretflow.preprocessing.scaler import MinMaxScaler
from secretflow.preprocessing.encoder import LabelEncoder
encoder = LabelEncoder()
data['job']=encoder.fit_transform(data['job'])
data['marital']=encoder.fit_transform(data['marital'])
data['education']= encoder.fit_transform(data['education'])
data['default']= encoder.fit_transform(data['default'])
data['housing']= encoder.fit_transform(data['housing'])
data['loan']=encoder.fit_transform(data['loan'])
data['contact']=encoder.fit_transform(data['contact'])
data['poutcome']= encoder.fit_transform(data['poutcome'])
data['month']=encoder.fit_transform(data['month'])
label = encoder.fit_transform(label)
print(f"label= {type(label)},\ndata = {type(data)}")

label= <class 'secretflow.data.vertical.dataframe.VDataFrame'>,
data = <class 'secretflow.data.vertical.dataframe.VDataFrame'>


In [12]:
from secretflow.data.split import train_test_split
random_state=1234
train_data,test_data=train_test_split(
    data,train_size=0.8,random_state=random_state
)
train_label,test_label= train_test_split(
    label,train_size=0.8,random_state=random_state
)

# 3 联邦模型

## 3.1 基础模型创建

In [13]:
def create_base_model(input_dim, output_dim, name='base_model'):
    #Create model
    def create_model():
        from tensorflow import keras
        from tensorflow.keras import layers
        import tensorflow as tf
        # base model config
        model = keras.Sequential(
            [
                keras.Input(shape=input_dim),
                layers.Dense(100,activation="relu"),
                layers.Dense(output_dim, activation="relu")
            ]
        )
        # Compile model
        model.summary()
        model.compile(
            # 模型分布在各方，需要compile才能前向后向传播
            loss='binary_crossentropy',
            optimizer='adam',
            metrics=["accuracy",tf.keras.metrics.AUC()]
        )
        return model
    return create_model

In [14]:
# prepare model
hidden_size = 32
model_base_alice=create_base_model(4,hidden_size)
model_base_bob =create_base_model(12,hidden_size)
model_base_alice()
model_base_bob()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 100)               500       
                                                                 
 dense_7 (Dense)             (None, 32)                3232      
                                                                 
Total params: 3,732
Trainable params: 3,732
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 100)               1300      
                                                                 
 dense_9 (Dense)             (None, 32)                3232      
                                                                 
Total params: 4,532
Trainable 

In [15]:
def create_fuse_model(input_dim, output_dim, party_nums, name='fuse_model'):
    def create_model():
        from tensorflow import keras
        from tensorflow.keras import layers
        import tensorflow as tf
        # input
        input_layers =[]
        for i in range(party_nums):
            input_layers.append(
                keras.Input(
                    input_dim,
                )
            )
        merged_layer= layers.concatenate(input_layers)
        fuse_layer = layers.Dense(64,activation='relu')(merged_layer)
        output =layers.Dense(output_dim,activation='sigmoid')(fuse_layer)
        model = keras.Model(inputs=input_layers, outputs=output)
        model.summary()
        model.compile(
            loss='binary_crossentropy',
            optimizer='adam',
            metrics=[tf.keras.metrics.AUC()]
        )
        return model

    return create_model

In [16]:
model_fuse=create_fuse_model(input_dim=hidden_size, party_nums=2,output_dim=1)
model_fuse()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 32)]         0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 32)]         0           []                               
                                                                                                  
 concatenate_1 (Concatenate)    (None, 64)           0           ['input_3[0][0]',                
                                                                  'input_4[0][0]']                
                                                                                                  
 dense_10 (Dense)               (None, 64)           4160        ['concatenate_1[0][0]']    

## 3.2 创建拆分模型

In [17]:
base_model_dict ={alice: model_base_alice, bob: model_base_bob}

In [18]:
from secretflow.security.privacy import DPStrategy, LabelDP
from secretflow.security.privacy.mechanism.tensorflow import GaussianEmbeddingDP

# Define DP operations
train_batch_size = 128
gaussian_embedding_dp = GaussianEmbeddingDP(
    noise_multiplier=0.5,
    l2_norm_clip=1.0,
    batch_size=train_batch_size,
    num_samples=train_data.values.partition_shape()[alice][0],
    is_secure_generator=False,
)
label_dp = LabelDP(eps=64.0)
dp_strategy_alice = DPStrategy(label_dp=label_dp)
dp_strategy_bob = DPStrategy(embedding_dp=gaussian_embedding_dp)
dp_strategy_dict = {alice: dp_strategy_alice, bob: dp_strategy_bob}
dp_spent_step_freq = 10

In [19]:
sl_model = SLModel(
    base_model_dict=base_model_dict,
    device_y=alice,
    model_fuse=model_fuse,
    dp_strategy_dict=dp_strategy_dict
)

INFO:root:Create proxy actor <class 'secretflow.ml.nn.sl.backend.tensorflow.sl_base.PYUSLTFModel'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.ml.nn.sl.backend.tensorflow.sl_base.PYUSLTFModel'> with party bob.


In [20]:
sf.reveal(test_data.partitions[alice].data)

,age,job,marital,education
1426,38,2,1,1
416,31,7,2,1
3977,37,0,1,2
2291,42,0,1,1
257,28,10,2,1
...,...,...,...,...
1508,37,9,2,1
979,56,1,0,0
3494,50,1,1,0
42,52,1,1,1


In [21]:
sf.reveal(test_label.partitions[alice].data)


,y
1426,0
416,0
3977,0
2291,0
257,0
...,...
1508,0
979,0
3494,0
42,0


In [22]:
history = sl_model.fit(
    train_data,
    train_label,
    validation_data=(test_data, test_label),
    epochs=10,
    batch_size=train_batch_size,
    shuffle=True,
    verbose=1,
    validation_freq=1,
    dp_spent_step_freq=dp_spent_step_freq,
)

INFO:root:SL Train Params: {'self': <secretflow.ml.nn.sl.sl_model.SLModel object at 0x7f2bd7f443a0>, 'x': VDataFrame(partitions={PYURuntime(alice): <secretflow.data.core.partition.Partition object at 0x7f2b542e3790>, PYURuntime(bob): <secretflow.data.core.partition.Partition object at 0x7f2b54377040>}, aligned=True), 'y': VDataFrame(partitions={PYURuntime(alice): <secretflow.data.core.partition.Partition object at 0x7f2b54377e50>}, aligned=True), 'batch_size': 128, 'epochs': 10, 'verbose': 1, 'callbacks': None, 'validation_data': (VDataFrame(partitions={PYURuntime(alice): <secretflow.data.core.partition.Partition object at 0x7f2b54376e60>, PYURuntime(bob): <secretflow.data.core.partition.Partition object at 0x7f2b54377cd0>}, aligned=True), VDataFrame(partitions={PYURuntime(alice): <secretflow.data.core.partition.Partition object at 0x7f2b54376590>}, aligned=True)), 'shuffle': True, 'sample_weight': None, 'validation_freq': 1, 'dp_spent_step_freq': 10, 'dataset_builder': None, 'audit_lo

Epoch 1/10


Train Processing: :   7%|▋         | 2/29 [00:00<00:02, 13.19it/s]

Epoch 2/10


Train Processing: :  10%|█         | 3/29 [00:00<00:01, 20.13it/s]

...

Train Processing: :  14%|█▍        | 4/29 [00:00<00:00, 35.00it/s]

Epoch 9/10


Train Processing: :  14%|█▍        | 4/29 [00:00<00:00, 37.41it/s]

Epoch 10/10


Train Processing: :  97%|█████████▋| 28/29 [00:00<00:00, 41.62it/s, {'train_loss': 0.3254059, 'train_auc_1': 0.7217264, 'val_loss': 0.3530417, 'val_auc_1': 0.705366}]


In [23]:
global_metric = sl_model.evaluate(test_data, test_label, batch_size=128)

Evaluate Processing: :  88%|████████▊ | 7/8 [00:00<00:00, 154.92it/s, loss=0.359, auc_1=0.686]


In [24]:
base_model_path={
    alice:"./alice_base_model",
    bob:"./bob_base_model"
}
fuse_model_path='./fuse_model'
sl_model.save_model(
    base_model_path=base_model_path,
    fuse_model_path=fuse_model_path
)

In [25]:
reload_base_model_dict={
    alice:None,
    bob:None
}

reload_sl_model=SLModel(
    base_model_dict=reload_base_model_dict,
    device_y=alice
)
reload_sl_model.load_model(
    base_model_path=base_model_path,
    fuse_model_path=fuse_model_path
)

INFO:root:Create proxy actor <class 'secretflow.ml.nn.sl.backend.tensorflow.sl_base.PYUSLTFModel'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.ml.nn.sl.backend.tensorflow.sl_base.PYUSLTFModel'> with party bob.


In [26]:
metrics=reload_sl_model.evaluate(test_data,test_label, batch_size=128)

Evaluate Processing: :  88%|████████▊ | 7/8 [00:00<00:00, 15.68it/s, loss=3.52, auc_1=0.753]


# 4. 单方建模（对比）

In [27]:
def create_single_model():
    model = keras.Sequential(
        [
            keras.Input(shape=4),
            layers.Dense(100, activation="relu"),
            layers.Dense(64, activation='relu'),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='sigmoid'),
        ]
    )
    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=["accuracy", tf.keras.metrics.AUC()],
    )
    return model


single_model = create_single_model()

In [28]:
type(alice_data)

numpy.ndarray

In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
alice_data = df[["age","job","marital","education","y"]]
print(type(alice_data))
single_part_data = alice_data.copy()
single_part_data.loc[:,'job'] = encoder.fit_transform(alice_data['job'])
single_part_data.loc[:,'marital'] = encoder.fit_transform(alice_data['marital'])
single_part_data.loc[:,'education'] = encoder.fit_transform(alice_data['education'])
single_part_data.loc[:,'y'] = encoder.fit_transform(alice_data['y'])

<class 'pandas.core.frame.DataFrame'>


In [30]:
y = single_part_data['y']
alice_data = single_part_data.drop(columns=['y'], inplace=False)

In [31]:
scaler = MinMaxScaler()
alice_data = scaler.fit_transform(alice_data)

In [32]:
train_data, test_data = train_test_split(
    alice_data, train_size=0.8, random_state=random_state
)
train_label, test_label = train_test_split(y, train_size=0.8, random_state=random_state)

In [33]:
alice_data.shape

(4521, 4)

In [34]:
single_model.fit(
    train_data,
    train_label,
    validation_data=(test_data, test_label),
    batch_size=128,
    epochs=10,
    shuffle=False,
)

Epoch 1/10
29/29 [==============================] - 1s 8ms/step - loss: 0.5426 - accuracy: 0.8415 - auc_4: 0.4500 - val_loss: 0.4019 - val_accuracy: 0.8729 - val_auc_4: 0.4027
Epoch 2/10
29/29 [==============================] - 0s 3ms/step - loss: 0.3710 - accuracy: 0.8877 - auc_4: 0.4501 - val_loss: 0.3967 - val_accuracy: 0.8729 - val_auc_4: 0.3968
Epoch 3/10
29/29 [==============================] - 0s 2ms/step - loss: 0.3642 - accuracy: 0.8877 - auc_4: 0.4264 - val_loss: 0.3929 - val_accuracy: 0.8729 - val_auc_4: 0.4029
Epoch 4/10
29/29 [==============================] - 0s 2ms/step - loss: 0.3596 - accuracy: 0.8877 - auc_4: 0.4464 - val_loss: 0.3899 - val_accuracy: 0.8729 - val_auc_4: 0.4239
Epoch 5/10
29/29 [==============================] - 0s 2ms/step - loss: 0.3587 - accuracy: 0.8877 - auc_4: 0.4591 - val_loss: 0.3878 - val_accuracy: 0.8729 - val_auc_4: 0.4458
Epoch 6/10
29/29 [==============================] - 0s 2ms/step - loss: 0.3574 - accuracy: 0.8877 - auc_4: 0.4735 - val_

In [35]:
single_model.evaluate(test_data, test_label, batch_size=128)

8/8 [==============================] - 0s 2ms/step - loss: 0.3806 - accuracy: 0.8729 - auc_4: 0.5371


[0.38060322403907776, 0.8729282021522522, 0.5370941162109375]